In [ ]:
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
%matplotlib inline

# Datos Población

In [ ]:
poblacion_df = pd.read_csv( './data/población.csv', sep = ';', engine = 'python' )

In [ ]:
poblacion_df.dtypes

In [ ]:
poblacion_df.head()

In [ ]:
poblacion_df = poblacion_df.set_index( 'Ciudad' ).stack().reset_index().rename( columns = { 'level_1' : 'Año', 0 : 'Población' } )

In [ ]:
poblacion_df.head()

# Accidents dataset

In [ ]:
accidents_2017_df = pd.read_csv( './Registro_Nacional_de_accidentalidad_2017.csv' )
accidents_2016_df = pd.read_csv( './Registro_Nacional_de_accidentalidad_2016.csv' )
accidents_2015_df = pd.read_csv( './Registro_Nacional_de_accidentalidad_2015.csv' )
accidents_2014_df = pd.read_csv( './Registro_Nacional_de_accidentalidad_2014.csv' )
accidents_2013_df = pd.read_csv( './Registro_Nacional_de_accidentalidad_2013.csv' )

In [ ]:
print( accidents_2017_df.shape )
print( accidents_2016_df.shape )
print( accidents_2015_df.shape )
print( accidents_2014_df.shape )
print( accidents_2013_df.shape )

In [ ]:
accidents_df = pd.concat( [ accidents_2013_df,
                           accidents_2014_df,
                           accidents_2015_df,
                           accidents_2016_df,
                           accidents_2017_df ] )

In [ ]:
pandas_profiling.ProfileReport( accidents_df )

In [ ]:
#accidentalidad_df[ 'Ciudad de Registro de IPAT' ].value_counts( dropna = False ).head( 20 )

In [ ]:
accidentalidad_df = accidentalidad_df.loc[ accidentalidad_df[ 'Ciudad de Registro de IPAT' ].isin( ['BOGOTA', 'MEDELLIN', 'CALI', 'BARRANQUILLA' ] ) ]

In [ ]:
#accidentalidad_df[ 'Fecha' ] = pd.to_datetime( ( accidentalidad_df[ 'Año Accidente' ] + '-' +
#                                               accidentalidad_df[ 'Mes Accidente' ] + '-' +
#                                               accidentalidad_df[ 'Dia Accidente' ] ), format='%Y-%m-%d' )

In [ ]:
#del accidentalidad_df[ 'Año Accidente' ]
del accidentalidad_df[ 'Mes Accidente' ]
del accidentalidad_df[ 'Dia Accidente' ]
del accidentalidad_df[ 'Area del Accidente' ]
del accidentalidad_df[ 'Clase de Accidente' ]
del accidentalidad_df[ 'Clase de Choque' ]
del accidentalidad_df[ 'Gravedad del Accidente' ]

In [ ]:
accidentalidad_df.rename( columns = { 'Ciudad de Registro de IPAT' : 'Ciudad', 'Año Accidente' : 'Año' }, inplace = True )

In [ ]:
accidents_df.dtypes

In [ ]:
#accidentalidad_df[ 'Fecha' ] = accidentalidad_df[ 'Fecha' ].apply( lambda x: x.replace( day = 1 ) )

In [ ]:
accidents_df.head()

In [ ]:
accidentalidad_df[ 'Cantidad' ] = 1

In [ ]:
grouped_accidentalidad_df = accidentalidad_df.groupby( [ accidentalidad_df[ 'Ciudad' ], accidentalidad_df[ 'Año' ] ] ).agg( { 'Cantidad' : 'sum' } ).reset_index()

In [ ]:
grouped_accidentalidad_df.dtypes

In [ ]:
grouped_accidentalidad_df.head()

In [ ]:
grouped_accidentalidad_df.shape

# Normalización

In [ ]:
grouped_accidentalidad_df = grouped_accidentalidad_df.merge( poblacion_df, how = 'left', on = [ 'Ciudad', 'Año' ] )

In [ ]:
def cantidad_norm( x ):
    return x[ 'Cantidad' ] * 100000 / x[ 'Población' ]

grouped_accidentalidad_df[ 'Cantidad_norm' ] = grouped_accidentalidad_df.apply( lambda x: cantidad_norm( x ), axis = 1 )

In [ ]:
#del grouped_accidentalidad_df[ '2013' ]
#del grouped_accidentalidad_df[ '2014' ]
#del grouped_accidentalidad_df[ '2015' ]
#del grouped_accidentalidad_df[ '2016' ]
#del grouped_accidentalidad_df[ '2017' ]

In [ ]:
grouped_accidentalidad_df.head()

# Unstacking

In [ ]:
grouped_accidentalidad_df = grouped_accidentalidad_df[ [ 'Ciudad', 'Año', 'Cantidad_norm' ] ].set_index( [ 'Año', 'Ciudad' ] ).unstack( level = -1 )

In [ ]:
grouped_accidentalidad_df.columns = grouped_accidentalidad_df.columns.map( '_'.join )

In [ ]:
grouped_accidentalidad_df.rename( columns = { 'Cantidad_norm_BARRANQUILLA' : 'BARRANQUILLA', 
                                            'Cantidad_norm_BOGOTA' : 'BOGOTA',
                                            'Cantidad_norm_CALI' : 'CALI',
                                            'Cantidad_norm_MEDELLIN' : 'MEDELLIN' }, inplace = True )

In [ ]:
grouped_accidentalidad_df = grouped_accidentalidad_df.reset_index()

In [ ]:
grouped_accidentalidad_df.head()

In [ ]:
grouped_accidentalidad_df.to_csv( './data/accidentalidad.csv', index = False, sep = ',' )